In [1]:
import pandas as pd
import lxml.html as lh
import io
import requests
from bs4 import BeautifulSoup
import re
import itertools
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import json
import os

In [2]:
def scrapeTranscriptFormat1(url, sep):
    html = requests.get(url)
    html = html.text
    bs = BeautifulSoup(html, "lxml")
    paragraphs = bs.findAll("p")

    # remove 'u' tags embedded within each paragraph tag
    for paragraph in paragraphs:
        try:
            paragraph.find('u').decompose()
        except:
            continue

    # Retrieve speaker and speech from each paragraph tag
    speaker = []
    speech = []
    pattern = r'\[.*?\]'

    for paragraph in paragraphs:
        try:
            speechText = paragraph.text.replace(u'\xa0', u'') # remove space
            speechText = re.sub(pattern, '', speechText) # remove brackets and their contents
#             print(re.search("[0-9]{2}\)[\r\n]+(.*)", speechText).group(1))
#             print(speechText)
            if sep == "parenthesis":
                speech.append(re.search("[0-9]{2}\)[\r\n]+(.*)", speechText).group(1).strip(" ")) 
            else:
                speech.append(re.search(":[\r\n]+(.*)", speechText).group(1).strip(" ")) # search for speaker's speech, append to list
            speaker.append(re.search("^(.*?):", speechText).group(1)) # search for speaker name, append to list            
        except:
            continue
#     print(len(speaker))
#     print(len(speech))
    # Convert to dataframe
    return pd.DataFrame({'name': speaker, 'speech': speech})

In [3]:
deb = pd.read_csv('debates.csv')
deb

,ref,cand,date,url
0,biden-trump debate,both,9/29,https://www.rev.com/blog/transcripts/donald-tr...
1,biden-cleveland speech,biden,9/30,https://www.rev.com/blog/transcripts/joe-biden...
2,biden-johnstown speech,biden,9/30,https://www.rev.com/blog/transcripts/joe-biden...
3,biden-michigan speech,biden,10/2,https://www.rev.com/blog/transcripts/joe-biden...
4,biden-virtual speech,biden,10/3,https://www.rev.com/blog/transcripts/joe-biden...
5,biden-miami speech,biden,10/5,https://www.rev.com/blog/transcripts/joe-biden...
6,biden-getty-speech,biden,10/6,https://www.rev.com/blog/transcripts/joe-biden...
7,biden-vegas-visit,biden,10/9,https://www.rev.com/blog/transcripts/joe-biden...
8,biden-erie-speech,biden,10/10,https://www.rev.com/blog/transcripts/joe-biden...
9,biden-toledo-speech,biden,10/12,https://www.rev.com/blog/transcripts/joe-biden...


In [4]:
dfs = []
for index, row in deb.iterrows():
    dfs.append(scrapeTranscriptFormat1(row["url"], "parenthesis"))

In [5]:
pd.concat(dfs)

,name,speech
0,Chris Wallace,Good evening from the Health Education Campus ...
1,Chris Wallace,This debate is being conducted under health an...
2,Vice President Joe Biden,"How you doing, man?"
3,President Donald J. Trump,How are you doing?
4,Vice President Joe Biden,I’m well.
...,...,...
400,Savannah Guthrie,There are people who want to know why they sho...
401,President Trump,Because I’ve done a great job. We have the str...
402,Savannah Guthrie,I got to leave it there. I got a wrap from the...
403,President Trump,Thank you very much.


In [6]:
pd.concat(dfs).to_csv('debates_transcripts.csv', index=False)